In [1]:
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("libgsl.dylib")
#pragma cling load("libG4processes.dylib")
#include <iostream>
#include "G4LegendrePolynomial.hh"
#include <random>
#include <cstdlib>
#include "gsl/gsl_sf_legendre.h"
#include <stdio.h> 
#include <stdlib.h> 
#include <chrono>



{
    G4LegendrePolynomial lp;
    gsl_sf_legendre_t norm = GSL_SF_LEGENDRE_NONE;
    std::random_device rd;
    std::mt19937 mt(rd());
    std::uniform_real_distribution<double> xdist(-1, 1);
    
    std::cout << "L,  " << " t_gsl_s, " << "t_g4_s, " << "avg_diff2" << std::endl;    
    

       for (int l=0; l < 10; ++l){
           double gsl_acc_time_ns = 0;
           double g4_acc_time_ns = 0;
           double sum_diff2 = 0;
           double avg_diff2 = 0;
           
           std::vector<double> result_array(gsl_sf_legendre_array_n(l), 0);
           
           
           for (int i=0; i < 1000000; ++i){
                double x = xdist(mt);
                
                auto started_gsl = std::chrono::high_resolution_clock::now();
                gsl_sf_legendre_array_e(norm, l, x, -1, &(result_array[0]));
                auto done_gsl = std::chrono::high_resolution_clock::now(); 
                 
                gsl_acc_time_ns += std::chrono::duration_cast<std::chrono::nanoseconds>(done_gsl-started_gsl).count();
                
                for (int m = 0; m <= l; ++m){
                    auto started_g4 = std::chrono::high_resolution_clock::now();
                    double alpg4 = lp.EvalAssocLegendrePoly(l, m, x); 
                    auto done_g4 = std::chrono::high_resolution_clock::now(); 
                    g4_acc_time_ns += std::chrono::duration_cast<std::chrono::nanoseconds>(done_g4-started_g4).count(); 
                    
               
                    double alp_gsl = result_array[gsl_sf_legendre_array_index(l, m)] ;
                    sum_diff2 += pow(alp_gsl - alpg4, 2);
                    avg_diff2 = sqrt(sum_diff2 * 1e-18/(2000*l+1000));
                      
               } // loop over m
           } // loop over i
     
            std::cout << l << " , " <<  gsl_acc_time_ns * 1e-9 << " , " << g4_acc_time_ns * 1e-9 << " , " << avg_diff2 << std::endl;
            
       } // loop over l

}
   

L,   t_gsl_s, t_g4_s, avg_diff2
0 , 0.0390262 , 0.0290322 , 0
1 , 0.0497461 , 0.0726489 , 1.03909e-24
2 , 0.0562754 , 0.115281 , 4.80259e-24
3 , 0.0721993 , 0.159308 , 2.22666e-23
4 , 0.0889937 , 0.226623 , 1.65148e-22
5 , 0.106134 , 0.425902 , 1.9193e-20
6 , 0.145657 , 0.698448 , 1.18497e-19
7 , 0.191608 , 1.17941 , 1.04837e-18
8 , 0.23956 , 1.86927 , 5.64144e-18
9 , 0.263479 , 2.65356 , 2.98868e-16
